<a href="https://colab.research.google.com/github/funpi89/colab_tensorflow2.0/blob/master/Preparing_a_Tensorflow_model_for_a_mobile_device_with_Tensorflow_Lite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing dependencies and setting up the evironment

In [0]:
!pip install tensorflow-gpu==1.13.1

     |████████████████████████████████| 345.2MB 79kB/s 
     |████████████████████████████████| 3.2MB 50.9MB/s 
     |████████████████████████████████| 368kB 56.6MB/s 
ERROR: tensorflow 1.14.0 has requirement tensorboard<1.15.0,>=1.14.0, but you'll have tensorboard 1.13.1 which is incompatible.
ERROR: tensorflow 1.14.0 has requirement tensorflow-estimator<1.15.0rc0,>=1.14.0rc0, but you'll have tensorflow-estimator 1.13.0 which is incompatible.
  Found existing installation: tensorboard 1.14.0
    Uninstalling tensorboard-1.14.0:
      Successfully uninstalled tensorboard-1.14.0
  Found existing installation: tensorflow-estimator 1.14.0
    Uninstalling tensorflow-estimator-1.14.0:
      Successfully uninstalled tensorflow-estimator-1.14.0


# Import project dependencies

In [0]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import fashion_mnist

tf.__version__

'1.13.1'

# Dataset preprocessing

In [0]:
# loading the dataset

(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [0]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [0]:
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)

# Building the model

In [0]:
model = tf.keras.models.Sequential()

In [0]:
model.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(units=10, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

In [0]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 6s 106us/sample - loss: 0.5306 - sparse_categorical_accuracy: 0.8119
Epoch 2/5
60000/60000 [==============================] - 5s 88us/sample - loss: 0.4008 - sparse_categorical_accuracy: 0.8560
Epoch 3/5
60000/60000 [==============================] - 5s 88us/sample - loss: 0.3672 - sparse_categorical_accuracy: 0.8666
Epoch 4/5
60000/60000 [==============================] - 5s 89us/sample - loss: 0.3465 - sparse_categorical_accuracy: 0.8730
Epoch 5/5
60000/60000 [==============================] - 5s 88us/sample - loss: 0.3304 - sparse_categorical_accuracy: 0.8783


In [0]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)

10000/10000 [==============================] - 0s 49us/sample - loss: 0.3461 - sparse_categorical_accuracy: 0.8768


In [0]:
print("Test accuracy : {}".format(test_accuracy))

Test accuracy : 0.876800000667572


# Saving the model

In [0]:
model_name = "fashion_mobile_model.h5"
tf.keras.models.save_model(model, model_name)

# Converting the model into Tensorflow Lite version

In [0]:
# creating tensorflow lite Converter
converter = tf.lite.TFLiteConverter.from_keras_model_file(model_name)

Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 4 variables.
INFO:tensorflow:Converted 4 variables to const ops.


In [0]:
# converting the model
tflite_model = converter.convert()

In [0]:
# saving the TFLite version of the model
with open("tf_model.tflite","wb") as f:
  f.write(tflite_model)